In [306]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor

In [307]:
## Generate regression samples
n_points = 100
X, y = make_regression(n_samples=n_points, random_state=12, n_features=20)

In [308]:
## Defining squared loss
def sq_loss(y_true, y_pred):
  N = len(y_true)
  loss = 0
  for i in range(N):
    loss += (y_true[i] - y_pred[i])**2
  return loss/N

## Function to initialize the first model using gradient decent
def initial_model(y_true, gaama, epoch, lr):
  N = len(y_true)
  for e in range(epoch):
    dL = 0
    for i in range(N):
      dL += ( -2 * (y_true[i] - gaama) )
    gaama -= lr * dL/N
  return gaama

In [309]:
M = 300 ## no. of base learners
gaama = 0 ## initial value for model_0
epoch = 100
lr = 0.1 ## learning rate

####### Implementation of GBDT regressor #######

## Initialize the 0th model
models_list = []
model_0 = initial_model(y, gaama, epoch, lr)
models_list.append(model_0)

## Build the models addtively 
for m in range(M):
  ## Compute fx i.e predection from previous model 
  if m == 0:
    fx = [model_0 for k in range(n_points)]
  else:
    fx = pred(X,m+1)

  ## Coumpute suso residuals
  sudo_residuals = []
  for i in range(n_points):
    sudo_residuals.append((2/n_points) * (y[i] - fx[i]))

  ## Build model on residuals of previous predictions
  reg = DecisionTreeRegressor(random_state=0)
  reg.fit(X,sudo_residuals)
  models_list.append(reg)

In [310]:
## additive prediction  
def pred(X,M):
  res = models_list[0]
  for m in range(1,M):
    res += models_list[m].predict(X)
  return res

## Compute squared loss
print(f"Squared loss: {sq_loss(y, pred(X,M))}")

Squared loss: 0.17073166872050793
